[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/PySDM/blob/main/examples/PySDM_examples/Bolin_1958/timescales_comparison.ipynb)
[![launch on mybinder.org](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/PySDM.git/main?urlpath=lab/tree/examples/PySDM_examples/Bolin_1958/timescales_comparison.ipynb)
[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/PySDM/blob/main/examples/PySDM_examples/Bolin_1958/timescales_comparison.ipynb)

#### Timescales comparison 
##### Bolin
##### Gedzelman

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [4]:
import numpy as np
from PySDM.physics import si
from PySDM import Formulae

In [5]:
formulae = Formulae(
    terminal_velocity='RogersYau',
    drop_growth='Mason1951',
    diffusion_thermics='Neglect',
    saturation_vapour_pressure='AugustRocheMagnus',
    ventilation='Froessling1938',
    particle_shape_and_density='LiquidSpheres',
    air_dynamic_viscosity='ZografosEtAl1987',
    constants={'BOLIN_ISOTOPE_TIMESCALE_COEFF_C1': 1.63},
    isotope_relaxation_timescale='Bolin1958',
)
const = formulae.constants

TODO:
    - maybe use definitions from Bolin? As a .py file?

In [6]:
a =  radii = np.asarray([0.005, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.20]) * si.cm
# from Bolin
temperature = const.T0 + 10 * si.K
pressure = const.p_STP
D = formulae.diffusion_thermics.D(T=temperature, p=pressure) #diffusion coefficient
f = 1. #set
rho_s = 1
S = b = 0

In [7]:
tau = -4 * np.pi * a * D * f * rho_s * (S - 1)/(1 + b)